## Merge files

In [3]:
#imports
import pandas as pd
import numpy as np
import re

### Google merge and cleaning

In [4]:
#read test data from Google scraping
ny_goog = pd.read_csv("ny_goog.csv")
ca_goog = pd.read_csv("ca_goog.csv")
ch_goog = pd.read_csv("ch_goog.csv")
zagat = pd.read_csv("dc_google_zagat.csv")
post = pd.read_csv("post_goog.csv")

In [5]:
us_goog = pd.concat([ny_goog,ca_goog,ch_goog])
dc = pd.concat([zagat,post])

In [6]:
us_goog = us_goog.reset_index(drop=True)
dc = dc.reset_index(drop=True)

In [7]:
#drop description for this part
dc.drop("desc",axis=1,inplace=True)

In [8]:
us_goog = us_goog.applymap(lambda x: np.nan if x=="None" else x)
dc = dc.applymap(lambda x: np.nan if x=="None" else x)
#post = post.applymap(lambda x: np.nan if x=="None" else x)

In [9]:
#now check null values
us_goog.isnull().sum()

name      17
rev      113
price     76
dtype: int64

In [10]:
dc.isnull().sum()

name     78
price    90
rev      91
dtype: int64

In [11]:
post.isnull().sum()

name     0
rev      0
price    0
dtype: int64

In [12]:
#drop them
us_goog.dropna(inplace=True)
dc.dropna(inplace=True)
#post.dropna(inplace=True)

In [13]:
#let's clean the names
def cleanName(x):
    x = str(x).replace("\n", "")
    if x[-17:].lower()=='websitedirections':
        return x[:-17]
    else:
        return x

In [14]:
us_goog["name"] = us_goog["name"].apply(cleanName)
dc["name"] = dc["name"].apply(cleanName)
#post["name"] = post["name"].apply(cleanName)

In [15]:
#separate rev 
us_goog["rating"] = us_goog["rev"].apply(lambda x: x.split("\n")[0])
dc["rating"] = dc["rev"].apply(lambda x: x.split("\n")[0])
#post["rating"] = post["rev"].apply(lambda x: x.split("\n")[0])

In [16]:
#get the reviews if available
def getReviews(x):
    try:
        return x.split("\n")[1]
    except:
        return np.nan

In [17]:
us_goog["numrev"] = us_goog["rev"].apply(getReviews)
us_goog.dropna(inplace=True)

dc["numrev"] = dc["rev"].apply(getReviews)
dc.dropna(inplace=True)

# post["numrev"] = post["rev"].apply(getReviews)
# post.dropna(inplace=True)

In [18]:
#let's split up the price to price and cuisine
def getPrice(x):
    price = x.split("·")[0]
    if price[0] == "$":
        return price.replace(" ","")
    else:
        return np.nan

In [19]:
us_goog["goog_price"] = us_goog["price"].apply(getPrice)
us_goog.dropna(inplace=True)

dc["goog_price"] = dc["price"].apply(getPrice)
dc.dropna(inplace=True)

# post["goog_price"] = post["price"].apply(getPrice)
# post.dropna(inplace=True)

In [20]:
def getCuisine(x):
    try:
        cuisine = x.split("·")[1]
        if cuisine[0] != "$":
            if "Restaurant" in cuisine:
                return cuisine.replace("Restaurant","")
            else:
                return cuisine
        else:
            if x.split("·")[0][0] != "$": 
                return x.split("·")[0].replace("Restaurant","")
    except:
        return np.nan
        

In [21]:
us_goog["cuisine"] = us_goog["price"].apply(getCuisine)
us_goog.dropna(inplace=True)

dc["cuisine"] = dc["price"].apply(getCuisine)
dc.dropna(inplace=True)

# post["cuisine"] = post["price"].apply(getCuisine)
# post.dropna(inplace=True)

In [22]:
def cleanReviews(x):
    if re.findall(r"\d",x):
        return "".join(re.findall(r"\d",x))
    else:
        return np.nan

In [23]:
us_goog.numrev = us_goog["numrev"].apply(cleanReviews)
us_goog.dropna(inplace=True)

dc.numrev = dc["numrev"].apply(cleanReviews)
dc.dropna(inplace=True)

# post.numrev = post["numrev"].apply(cleanReviews)
# post.dropna(inplace=True)

In [24]:
us_goog.drop(["rev","price"],axis=1,inplace=True)
dc.drop(["rev","price"],axis=1,inplace=True)
#post.drop(["rev","price"],axis=1,inplace=True)

In [25]:
#save clean test data
us_goog.to_csv("us_goog.csv",encoding="utf-8",index=False)
dc.to_csv("dc_test.csv",encoding="utf-8",index=False)
# post.to_csv("post_test.csv",encoding="utf-8",index=False)

In [26]:
us_goog

,name,rating,numrev,goog_price,cuisine
0,Noodle Pudding,4.3,118,$$,Italian
1,Peter Luger Steak House,4.2,910,$$$$,Steak House
2,Vic's,4.1,43,$$,Mediterranean
3,Gnocco,3.9,72,$$,Pizza
4,Hasaki,4.2,63,$$,Japanese
5,Hearth,4.2,102,$$$,Fine Dining
6,Kanoyama,4.1,88,$$,Japanese
8,Soba-ya,4.1,193,$$,Japanese
9,Da Umberto,4.4,41,$$$,Italian
10,Rocking Horse Cafe,4.0,137,$$,Mexican


### Michelin and Google Merge

In [27]:
michelin = pd.read_csv("combined_clean.csv")

In [28]:
#let's get the US data only
def findCountry(x):
    if re.search("[9]{1}[0-9]{4}",x[-5:]):
        return "CA"
    elif re.search("[1]{1}[0-9]{4}",x[-5:]):
        return "NY"
    elif re.search("[6]{1}[0-9]{4}",x[-5:]):
        return "IL"
    else:
        return "NA"

In [29]:
michelin["state"] = michelin["address"].apply(findCountry)

In [30]:
us = michelin[(michelin["state"]=="NY")|(michelin["state"]=="CA")|(michelin["state"]=="IL")]

In [31]:
#merge on name keep left (michelin restaurants)
us_mg = us.merge(us_goog,how="left",on="name")

In [32]:
#drop nulls and reset index
us_mg.dropna(inplace=True)
us_mg.reset_index(drop=True,inplace=True)

In [33]:
#change the price to numeric instead of $$$$
us_mg.goog_price = us_mg.goog_price.map({"$":1,"$$":2,"$$$":3,"$$$$":4})
dc["goog_price"] = dc.goog_price.map({"$":1,"$$":2,"$$$":3,"$$$$":4})
#post["goog_price"] = post.goog_price.map({"$":1,"$$":2,"$$$":3,"$$$$":4})

In [34]:
def generalizeC(x):
    cuisines = {"Chinese":["dim","dimsum","sichuan","taiwanese","tea","teahouse","mandarin"],
                "Mexican":["taco"],"International":["eclectic"],"Latin American":["nuevo"],
                "Japanese":["sushi","ramen","yakitori"],
                "American":["diner","family","hamburger","cafe","newamerican","traditionalamerican"],
               "Steakhouse":["steakhouse"],
               "Contemporary":["finedining"],
               "Spanish":["tapasbar"]}
    
    y = x.replace(" ","").lower()
    for key,val in cuisines.iteritems():
        if y in val:
            return key
    return x

In [35]:
us_mg.cuisine = us_mg.cuisine.apply(generalizeC)
dc.cuisine = dc.cuisine.apply(generalizeC)
#post.cuisine = post.cuisine.apply(generalizeC)

In [36]:
def cleanCuisine(x):
    x = re.sub(r"^\s*", "", x)
    x = re.sub(r"\s*$","",x)
    
    if x != "":
        return x
    else:
        return np.nan

In [37]:
us_mg["cuisine"] = us_mg["cuisine"].apply(cleanCuisine)
us_mg.dropna(inplace=True)

dc["cuisine"] = dc["cuisine"].apply(cleanCuisine)
dc.dropna(inplace=True)

# post["cuisine"] = post["cuisine"].apply(cleanCuisine)
# post.dropna(inplace=True)

In [38]:
us_mg.numrev = pd.to_numeric(us_mg.numrev)
us_mg.rating = pd.to_numeric(us_mg.rating)

dc.numrev = pd.to_numeric(dc.numrev)
dc.rating = pd.to_numeric(dc.rating)

# post.numrev = pd.to_numeric(post.numrev)
# post.rating = pd.to_numeric(post.rating)

In [39]:
us_mg.stars.value_counts()

0.0    891
1.0    144
2.0     24
3.0     18
Name: stars, dtype: int64

In [40]:
us_mg["bstars"] = us_mg.stars.map({0.0:0,1.0:1,2.0:1,3.0:1})
us_mg["cstars"] = us_mg.stars.map({0.0:0,1.0:1,2.0:2,3.0:2})

In [41]:
price_cuisine = pd.DataFrame(us_mg.groupby(["cuisine","goog_price"]).usd_avg.median())
avgPrice = pd.DataFrame(us_mg.groupby("goog_price").usd_avg.median())

In [42]:
price_cuisine

usd_avg
cuisine         goog_price         
Afghani         2             37.00
African         2             18.50
American        1             37.00
                2             37.00
                3             37.00
                4            112.50
Argentinian     3             37.00
Asian           1             37.00
                2             37.00
                3             37.00
Asian Fusion    1             18.50
                2             27.75
                3             37.00
Australian      2             37.00
Austrian        2             37.00
                3             49.50
Bangladeshi     1             18.50
Bar             2             37.00
                3             37.00
                4            112.50
Bar & Grill     2             27.75
                3             37.00
                4             62.00
Barbecue        1             27.75
                2             37.00
Basque          2             18.50
                3             37.00
Belgian         3             37.00
Bistro          2             37.00
                3             37.00
...                             ...
Scandinavian    4            112.50
Seafood         2             37.00
                3             37.00
                4            112.50
Shanghainese    2             18.50
Small Plates    2             37.00
Soul Food       1             37.00
                2             18.50
Southeast Asian 1             18.50
                2             18.50
Southern  (US)  1             37.00
                2             37.00
Spanish         2             37.00
                3             37.00
Steakhouse      2             37.00
                3             62.00
                4            112.50
Tapas           2             37.00
                3             37.00
Thai            1             18.50
                2             37.00
                3             37.00
                4            112.50
Turkish         2             37.00
Vegetarian      2             18.50
                3             37.00
Vietnamese      1             37.00
                2             18.50
                3             49.50
Wine Bar        2             37.00

[167 rows x 1 columns]

In [43]:
ap = avgPrice.to_dict()
pc = price_cuisine.to_dict()

In [44]:
pc

{'usd_avg': {('Afghani', 2): 37.0,
  ('African', 2): 18.5,
  ('American', 1): 37.0,
  ('American', 2): 37.0,
  ('American', 3): 37.0,
  ('American', 4): 112.5,
  ('Argentinian', 3): 37.0,
  ('Asian', 1): 37.0,
  ('Asian', 2): 37.0,
  ('Asian', 3): 37.0,
  ('Asian Fusion', 1): 18.5,
  ('Asian Fusion', 2): 27.75,
  ('Asian Fusion', 3): 37.0,
  ('Australian', 2): 37.0,
  ('Austrian', 2): 37.0,
  ('Austrian', 3): 49.5,
  ('Bangladeshi', 1): 18.5,
  ('Bar', 2): 37.0,
  ('Bar', 3): 37.0,
  ('Bar', 4): 112.5,
  ('Bar & Grill', 2): 27.75,
  ('Bar & Grill', 3): 37.0,
  ('Bar & Grill', 4): 62.0,
  ('Barbecue', 1): 27.75,
  ('Barbecue', 2): 37.0,
  ('Basque', 2): 18.5,
  ('Basque', 3): 37.0,
  ('Belgian', 3): 37.0,
  ('Bistro', 2): 37.0,
  ('Bistro', 3): 37.0,
  ('Brazilian', 2): 37.0,
  ('Brazilian', 3): 62.0,
  ('Breakfast', 2): 18.5,
  ('British', 2): 37.0,
  ('British', 3): 62.0,
  ('Brunch', 2): 37.0,
  ('Brunch', 3): 37.0,
  ('Burmese', 2): 18.5,
  ('Cajun', 2): 37.0,
  ('Californian', 2): 

In [45]:
def getAvgPriceByCuisine(x,y):
    xy = (x,y)
    
    try:
        return pc["usd_avg"][xy]
    except:
        return ap["usd_avg"][y]

In [46]:
us_mg["pc"] = np.vectorize(getAvgPriceByCuisine)(us_mg["cuisine"],us_mg["goog_price"])
dc["pc"] = np.vectorize(getAvgPriceByCuisine)(dc["cuisine"],dc["goog_price"])
#post["pc"]= np.vectorize(getAvgPriceByCuisine)(post["cuisine"],post["goog_price"])

In [47]:
us_mg.dropna(inplace=True)
dc.dropna(inplace=True)
#post.dropna(inplace=True)

In [48]:
us_mg.drop_duplicates(inplace=True)
dc.drop_duplicates(inplace=True)
#post.drop_duplicates(inplace=True)

In [49]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(list(us_mg["cuisine"].values) + list(dc["cuisine"].values))

# le = LabelEncoder()
# le.fit(list(us_mg["cuisine"].values) + list(post["cuisine"].values))

LabelEncoder()

In [50]:
us_mg["cuisine"] = le.transform(us_mg["cuisine"].values)
dc["cuisine"] = le.transform(dc["cuisine"].values)

# us_mg["cuisine"] = le.transform(us_mg["cuisine"].values)
# post["cuisine"] = le.transform(post["cuisine"].values)

In [51]:
#save again
us_mg.to_csv("us_mg.csv",encoding="utf-8",index=False)
dc.to_csv("dc_test.csv",encoding="utf-8",index=False)
#post.to_csv("post_test.csv",encoding="utf-8",index=False)

## Modeling 

In [52]:
from sklearn.cross_validation import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,classification_report,roc_curve,auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [53]:
X = us_mg[["rating","numrev","pc","cuisine"]]
y = us_mg["cstars"]

In [54]:
X_scaled = preprocessing.scale(X)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,test_size=0.4)

In [56]:
baseline = y.value_counts()/len(y)
baseline

0    0.860520
1    0.106383
2    0.033097
Name: cstars, dtype: float64

In [57]:
cv = StratifiedKFold(y, n_folds=5,shuffle=True)

In [58]:
def score(model, name, myX,myy):
    s = cross_val_score(model, myX, myy, cv=cv)
    print "{} Score:\t{:0.3} ± {:0.3}".format(name, s.mean().round(3), s.std().round(3))

In [59]:
def fitAndPrint(model):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print confusion_matrix(y_test,y_pred)

In [60]:
lm = LogisticRegression()

In [61]:
score(lm,"Logistic Regression",X_scaled,y)

Logistic Regression Score:	0.881 ± 0.019


In [62]:
fitAndPrint(lm)

[[281   7   0]
 [ 25  13   0]
 [  1  12   0]]


In [64]:
X_dc = dc[["rating","numrev","pc","cuisine"]]
X_scaled_dc = preprocessing.scale(X_dc)

In [65]:
y_pred = lm.predict(X_scaled_dc)

In [66]:
count=0
for a,b in zip(dc["name"].values,y_pred):
    if b>=1:
        print a,b

Corduroy 1
Fogo de Chão Brazilian Steakhouse 1
Minibar 1
The Lafayette 1
Marcel's 1
Plume 1
Equinox 1
Obelisk 1
The Capital Grille 1
Sakedokoro Makoto 1
Matisse 1
Equinox 1
Marcel’s 1


In [962]:
dt = DecisionTreeClassifier(class_weight="balanced")

In [963]:
score(dt,"Decision Tree",X_scaled,y)

Decision Tree Score:	0.844 ± 0.015


In [964]:
fitAndPrint(dt)

[[266  26   3]
 [ 21  10   3]
 [  1   4   5]]


In [965]:
dt.fit(X_scaled,y)

DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [966]:
predictions_dt = dt.predict(X_scaled_dc)

In [967]:
for a,b in zip(dc["name"].values,predictions_dt):
    if b>=1:
        print a,b

Corduroy 1
Fogo de Chão Brazilian Steakhouse 1
Minibar 1
The Lafayette 2
Marcel's 2
SEI 1
BlackSalt 1
Oceanaire Seafood Room (DC) 1
Blue Duck Tavern 1
Restaurant Nora 1
BLT Steak 1
1789 Restaurant 1
Equinox 1
Obelisk 1
The Capital Grille 2
Bistrot Lepic & Wine Bar 1
Beuchert’s Saloon 1
Toro Toro 1
La Chaumiere 1
Flight Wine Bar 1
Charlie Palmer Steak 1
Al Dente Italian Restaurant D.C. 1
Mari Vanna DC 1
DC Coast 1
DGS Delicatessen 1
Room 11 1
Buck's Fishing & Camping 1
Matisse 2
Bearnaise 1
Jack Rose Dining Saloon 1
Pesce 1
Equinox 1
Masseria 1
Marcel’s 2


In [968]:
rf = RandomForestClassifier(class_weight="balanced")

In [969]:
score(rf,"Random Forest",X_scaled,y)

Random Forest Score:	0.866 ± 0.01


In [970]:
fitAndPrint(rf)

[[285   6   4]
 [ 20  10   4]
 [  4   2   4]]


In [971]:
for a,b in zip(X.columns.tolist(), rf.feature_importances_):
    print a, b

goog_price 0.182294313169
rating 0.149111562989
numrev 0.297420914428
pc 0.220458599752
cuisine 0.150714609663


In [972]:
rf.fit(X_scaled,y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [973]:
predictions = rf.predict(X_scaled_dc)

In [974]:
count=0
for a,b in zip(dc["name"].values,predictions):
    if b>=1:
        print a,b

Minibar 1
Marcel's 1
1789 Restaurant 1
Beuchert’s Saloon 1
Toro Toro 1
Charlie Palmer Steak 1
DC Coast 1
Meridian Pint 1
Matisse 1
Garrison 1
Marcel’s 1


In [975]:
ada = AdaBoostClassifier()

In [976]:
score(ada,"Adaptive Boosting",X_scaled,y)

Adaptive Boosting Score:	0.819 ± 0.054


In [977]:
fitAndPrint(ada)

[[272  19   4]
 [ 23   9   2]
 [  5   4   1]]


In [978]:
predictions = ada.predict(X_scaled_dc)

In [979]:
count=0
for a,b in zip(dc["name"].values,predictions):
    if b>=1:
        print a,b
        count+=1
print count

Minibar 2
The Lafayette 1
Zaytinya 1
Marcel's 1
Plume 2
Joe's Seafood, Prime Steak & Stone Crab 1
Restaurant Nora 1
G by Mike Isabella 1
Equinox 1
Obelisk 2
2 Amys 1
7th Hill Pizza - Palisades 1
Legal Sea Foods 1
Campono 1
Doi Moi 1
Eat the Rich 1
Hank's Oyster Bar 1
Oyamel Cocina Mexicana 1
Jaleo 1
La Chaumiere 1
Flight Wine Bar 1
TaKorean | The Yards DC 1
Al Dente Italian Restaurant D.C. 1
Lebanese Taverna - DC 1
Pi Pizzeria - DC 1
Luke's Lobster Penn Quarter 1
Red Apron Butcher 1
Horace & Dickie's Seafood 1
Southern Efficiency 1
DGS Delicatessen 1
Tacos El Chilango 1
Boundary Road 1
WE, THE PIZZA 1
Pizzeria Paradiso 1
Dino's Grotto 1
Buck's Fishing & Camping 1
Matisse 1
Senart's Oyster & Chop House 1
Bearnaise 1
Soupergirl 1
Martin's Tavern 1
Boss Shepherd's Restaurant 1
2Amys 1
Zaytinya 1
Hank’s Oyster Bar 1
Crane & Turtle 1
Centrolina 1
Equinox 1
Masseria 1
Marcel’s 1
50


In [980]:
gb = GradientBoostingClassifier()

In [981]:
score(gb,"Gradient Boosting",X_scaled,y)

Gradient Boosting Score:	0.876 ± 0.02


In [982]:
fitAndPrint(gb)

[[285   7   3]
 [ 21  11   2]
 [  3   3   4]]


In [983]:
predictions = gb.predict(X_scaled_dc)

In [984]:
count=0
for a,b in zip(dc["name"].values,predictions):
    if b>=1:
        print a,b

The Lafayette 2
Marcel's 1
Plume 2
The Oval Room 1
Al Tiramisu 1
Komi 1
Blue Duck Tavern 1
1789 Restaurant 1
Obelisk 2
Toki Underground 1
Brasserie Beck 1
Sakedokoro Makoto 1
Proof 1
Founding Farmers 1
Flight Wine Bar 1
Good Stuff Eatery 1
Charlie Palmer Steak 1
Burger Tap & Shake Foggy Bottom 1
Al Dente Italian Restaurant D.C. 1
Georgia Brown's Restaurant 1
the pig 1
Old Ebbitt Grill 1
Boundary Stone 1
Ben's Chili Bowl 1
Clyde's of Georgetown 1
Buck's Fishing & Camping 1
Matisse 2
The Grill Room 1
Jack Rose Dining Saloon 1
Pesce 1
Acacia Bistro & Wine Bar 1
Convivial 1
Maketto 1
Brasserie Beck 1
Garrison 1
Marcel’s 1


In [985]:
et = ExtraTreesClassifier()

In [986]:
score(et,"Extra Trees Classifier",X_scaled,y)

Extra Trees Classifier Score:	0.85 ± 0.02


In [987]:
fitAndPrint(et)

[[278  16   1]
 [ 22   8   4]
 [  4   1   5]]


In [988]:
predictions = gb.predict(X_scaled_dc)

In [989]:
count=0
for a,b in zip(dc["name"].values,predictions):
    if b>=1:
        print a,b

The Lafayette 2
Marcel's 1
Plume 2
The Oval Room 1
Al Tiramisu 1
Komi 1
Blue Duck Tavern 1
1789 Restaurant 1
Obelisk 2
Toki Underground 1
Brasserie Beck 1
Sakedokoro Makoto 1
Proof 1
Founding Farmers 1
Flight Wine Bar 1
Good Stuff Eatery 1
Charlie Palmer Steak 1
Burger Tap & Shake Foggy Bottom 1
Al Dente Italian Restaurant D.C. 1
Georgia Brown's Restaurant 1
the pig 1
Old Ebbitt Grill 1
Boundary Stone 1
Ben's Chili Bowl 1
Clyde's of Georgetown 1
Buck's Fishing & Camping 1
Matisse 2
The Grill Room 1
Jack Rose Dining Saloon 1
Pesce 1
Acacia Bistro & Wine Bar 1
Convivial 1
Maketto 1
Brasserie Beck 1
Garrison 1
Marcel’s 1


In [992]:
from sklearn.svm import SVC

In [993]:
sv = SVC(kernel="poly")

In [994]:
score(sv,"SVM",X_scaled,y)

SVM Score:	0.873 ± 0.009


In [995]:
fitAndPrint(sv)

[[282  13   0]
 [ 20  12   2]
 [  3   6   1]]


In [996]:
predictions = sv.predict(X_scaled_dc)

In [997]:
count=0
for a,b in zip(dc["name"].values,predictions):
    if b>=1:
        print a,b

Fogo de Chão Brazilian Steakhouse 1
Minibar 1
The Lafayette 1
Marcel's 1
Plume 1
Komi 1
Obelisk 2
The Capital Grille 2
Campono 2
Founding Farmers 1
Bub and Pop's 2
Matisse 2
Marcel’s 1


In [1000]:
us_mg["zip"] = us_mg.address.apply(lambda x: x[-5:])

In [1001]:
us_mg.to_csv("us_data.csv",encoding="utf-8",index=False)

In [1005]:
us_mg

,name,stars,review,cooking_type,address,usd_avg,state,rating,numrev,goog_price,cuisine,bstars,cstars,pc,zip
0,Noodle Pudding,0.0,With Dean Martin and Frank Sinatra rotating on...,Italian,"38 Henry St., New York 11201",37.0,NY,4.3,118,2,54,0,0,37.00,11201
1,Peter Luger,1.0,Venerated Peter Luger resides in an expansive ...,Steakhouse,"178 Broadway, New York 11211",112.5,NY,4.2,911,4,99,1,1,112.50,11211
2,Gnocco,0.0,"A casual cucina with a lovely back garden, Gno...",Italian,"337 E. 10th St., New York 10009",37.0,NY,3.9,72,2,82,0,0,37.00,10009
3,Hasaki,0.0,"Since the mid-eighties, this local darling has...",Japanese,"210 E. 9th St., New York 10003",37.0,NY,4.2,63,2,55,0,0,37.00,10003
4,Hearth,0.0,"After more than a decade delighting fans, Chef...",Mediterranean,"403 E. 12th St., New York 10009",62.0,NY,4.2,102,3,32,0,0,87.25,10009
5,Kanoyama,0.0,Offering an impressive lineup of excellent qua...,Japanese,"175 Second Ave., New York 10003",37.0,NY,4.1,88,2,55,0,0,37.00,10003
6,Rocking Horse Cafe,0.0,"A Chelsea classic since 1988, this Mexican caf...",Mexican,"182 Eighth Ave., New York 10011",37.0,NY,4.0,137,2,66,0,0,37.00,10011
7,Paradou,0.0,For a restaurant that refuses to take itself t...,French,"8 Little W. 12th St., New York 10014",37.0,NY,3.4,97,3,44,0,0,37.00,10014
8,BONDST,0.0,There's no denying that BONDST is still trendy...,Japanese,"6 Bond St., New York 10014",62.0,NY,4.2,131,3,55,0,0,62.00,10014
9,Gotham Bar and Grill,1.0,"Warm, personable, and genuine, Gotham Bar and ...",American,"12 E. 12th St., New York 10003",62.0,NY,4.5,377,4,2,1,1,112.50,10003
